In [8]:
#################################### IMPORTS ###################################################
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import tensorflow as tf
import datetime
from keras.backend.tensorflow_backend import set_session

In [9]:
##################### INICIALIZACAO DE VARIAVEIS ################################################
index_arquivo = ['preco', 'hr_int']#, 'preco_pon', 'qnt_soma', 'max', 'min', 'IND', 'ISP'] #index do arquivo
steps = [] # 9h04 -> 17h50 a cada 5 segundos 
epocas = 1000 #quantidade de vezes que vai rodar todos os dias
janela = 10 #janela de valores
n_variaveis = len(index_arquivo) #'preco', 'hr_int', 'preco_pon', 'qnt_soma', 'max', 'min', 'IND', 'ISP'
l = n_variaveis-1
n_mercado = n_variaveis*janela #inputs
n_estados = 2 #ncont e valor
n_neuronios = 216 #numero de neuronios da camada escondida
lim_cont = 5
n_saidas = 2*lim_cont+1   #numero de saidas da rede (compra, vende, segura)
custo = 1.06/2 #custo da operao
melhor_reward = 0


versao_arquivo = 2

carregar_pesos = True
carregar_epoca_epsilon = False
epoca_init = 0
if carregar_epoca_epsilon:
    file = open("./epoca_epsilon.txt", "r")
    valores = file.read().split(',')
    epoca_init = int(valores[0])
    epsilon = float(valores[1])
    file.close()
else:
    epsilon = 1. #valor de epsilon
epsilon_min = 0.01 #valor minimo de epsilon
epsilon_decay = (epsilon - epsilon_min) / (epocas - epoca_init) #o valor que vai retirado do epsilon por epoca

rewards = [0] #variavel para guardar rewards
plotx = [0] #variavel para guardar valores a serem plotados do eixo x

In [10]:
####################### LEITURA DOS DADOS #######################################################
dias = 220 # dias_treino - 220 dias de 284 ~ 77,5% dos dados para treino

caminho_arquivo = ('./consolidado.csv')
arquivo = pd.read_csv(caminho_arquivo) #le arquivo
inputs = arquivo[index_arquivo]
print(inputs['preco'][0:2])
if versao_arquivo == 1: #se quiser usar apenas os dias com IND e ISP
    inputs = inputs[inputs['IND'] != 0]
    arquivo = arquivo[arquivo['IND'] != 0]

dt = arquivo['dt'].values #cria coluna apenas dos dias

steps = []
ultimo_dia = 0
dias_para_rodar = [] #variavel para colocar os dias a serem rodados
j = 0
hr = []
h = 0

for i in range( 0, len(dt) ):    
    if (dt[i] != ultimo_dia):
        steps.append(i) #numero de linhas entre dias
        ultimo_dia = dt[i]
        dias_para_rodar.append(j) #numero do dia
        j += 1
        h = 0
    hr.append(h)
    h += 1
step_max = np.amax(hr)
batch = step_max + 1

#normalizacao dos dados
hr = hr/step_max
inputs['hr_int'] = hr
pmean = np.mean( inputs.loc[:dias*batch, inputs.columns[0]] ) #define valor minimo do preco  #step = 106
pstd = np.std( inputs.loc[:dias*batch, inputs.columns[0]] ) #define valor maximo do preco
for i in range( inputs.shape[1] ): #roda normalizo para todas as colunas
    imean = np.mean( inputs.loc[:dias*batch, inputs.columns[i]] ) #pega valor maximo
    istd = np.std( inputs.loc[:dias*batch, inputs.columns[i]] ) #pega valor minimo
    
    inputs.loc[:, inputs.columns[i]] = ( inputs.loc[:, inputs.columns[i]] - imean ) / istd #normaliza prs
    
mini_batch_size = int(dias*batch*0.1)

0    3907.5
1    3900.0
Name: preco, dtype: float64


D:\anaconda\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
dias_teste = int(len(dias_para_rodar)*0.2) #20% dos dados para validação

In [15]:
########################################   BIBLIOTECAS ####################################
import random
import numpy as np
from keras.models import Model
from keras.layers import Dense, Input, concatenate
from keras import optimizers
from keras import backend as K            #importa backend para clear_session()


##################### MODELO DQN ####################################################
class DQNAgent:
    ########################### INICIALIZA ###########################################
    def __init__(self, state_size, mkt_size, action_size, epsilon, janela, n_neuronios, n_variaveis):
        self.state_size = state_size
        self.mkt_size = mkt_size
        self.n_neuronios = n_neuronios
        self.action_size = action_size
        self.n_variaveis = n_variaveis
        self.limpa_memoria_dia()
        self.limpa_memoria_epoca()
        self.gamma = 0.97       # discount rate
        self.epsilon = epsilon  # exploration rate
        self.learning_rate = 1e-4
        self.model = self.cria_modelo()
        self.model.summary()
        self.state = []
        self.next_state = []
        self.janela_precos = []
        self.prox_janela_precos = []

################################# REDE NEURAL ###########################################
    def cria_modelo(self):
        # Neural Net for Deep-Q learning Model
        #inputs
        x1 = Input(shape=(self.state_size,))
        x2 = Input(shape=(self.mkt_size,))
        
        #state hidden layers
        a1 = Dense(320)(x1)
        a2 = Dense(256)(a1)
        a3 = Dense(256)(a2)
        
        #mkt hidden layers
        b1 = Dense(320)(x2)
        b2 = Dense(256)(b1)
        b3 = Dense(256)(b2)
        
        #concatenacao
        x12 = concatenate([a3, b3], axis=-1)
        
        #camada de saida
        y = Dense(self.action_size, activation='linear')(x12)
        
        model = Model(inputs=[x1, x2], outputs=y)
        
        sgd = optimizers.SGD(lr=self.learning_rate, decay=1e-6, momentum=0.9, nesterov=True)
        model.compile(loss='mean_squared_error', optimizer=sgd)
        return model

    def limpa_memoria_dia(self):
        self.janela_precos = np.zeros(self.mkt_size)
        self.prox_janela_precos = np.zeros(self.mkt_size)
        self.state = []
        self.next_state = []
    
    def limpa_memoria_epoca(self):
        self.memory = []
        
    def toma_acao(self):    
        act_values = self.model.predict([self.state, self.janela_precos]) #calcula qual a melhor acao
        return np.argmax(act_values[0])  # returns action
    
    def treina_modelo(self, batch_size=mini_batch_size):
        minibatch = random.sample(self.memory, batch_size)
        
        x1 = np.zeros((batch_size, self.state_size))
        x2 = np.zeros((batch_size, self.mkt_size))
        y = np.zeros((batch_size, self.action_size))
        i = 0
        for acao, reward, estado, mercado, prox_estado, prox_mercado, done in minibatch:        
            
            target = reward
            if not done:
                target = (reward + self.gamma * np.amax(self.model.predict([prox_estado, prox_mercado])[0])) #pega valor que quer chegar

            target_f = self.model.predict([estado, mercado]) #pega valor que chegou
            target_f[0][acao+lim_cont] = target #define o valor que deseja chegar
            x1[i,:] = estado
            x2[i,:] = mercado
            y[i,:] = target_f
            i = i + 1
            
        self.model.fit([x1, x2], y, epochs=1, verbose=0) #treina modelo
            
    def remember(self, acao, reward, done):
        self.memory.append((acao, reward, self.state, self.janela_precos, self.next_state, self.prox_janela_precos, done))
    
    def carrega_pesos(self, name):
        self.model.load_weights(name) #carrega pesos

    def salva_pesos(self, name):
        self.model.save_weights(name) #salva pesos
                
########################  DECLARA MODELO ################################
modelo = DQNAgent(n_estados, n_mercado, n_saidas, epsilon, janela, n_neuronios, n_variaveis)

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 2)            0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
dense_13 (Dense)                (None, 320)          960         input_5[0][0]                    
__________________________________________________________________________________________________
dense_16 (Dense)                (None, 320)          6720        input_6[0][0]                    
____________________________________________________________________________________________

In [16]:
########################### FUNCOES ###############################################################

def atuacao( preco, ncont, acao, custo, valor ):  #preo atual, n de contratos posicionados,
                                                #acaoo atual, custo, valor da posio
    preco_cheio = 0.
    valor_cheio = 0.
    ncont_anterior = ncont #salva posio anterior
    ncont = acao #posicao atual = acao
    reward = 0.
    posicao = 0.
    dp = 0.

    
    #variaveis para tratamento do sinal de ncont, ncont_anterior e da acao
    acao = ncont - ncont_anterior  #ajuda a manter as condições anteriores
    var1 = ncont*ncont_anterior
    var2 = ncont*acao

    if acao != 0:
        #realizacao de lucros e custos
        if ((var2>0) and (var1>=0)):
            #aumento da posicao
            reward = -custo*abs(acao)    #reward = -custo*acao se houve operacao (aumento da posicao)
            #aumento da posicao = preco medio:
            valor = (ncont_anterior*valor + acao*preco)/ncont  
            
        else:
            valor_cheio = valor * pstd + pmean  #valores nao normalizados
            preco_cheio = preco * pstd + pmean
            if (var1)>= 0:
                #diminuicao da posicao
                reward = acao*(valor_cheio - preco_cheio)*10 - custo*abs(acao)  #reward se houve diminuicao da posicao
                #valor medio nao muda na diminuicao, somente se ncont==0:
                if ncont==0: valor=-10
                    
            else:
                #troca de posicao
                reward = (-ncont_anterior)*(valor_cheio - preco_cheio)*10 - custo*abs(acao)  #reward se houve troca da posicao
                #preco medio da posicao = preco da troca de posicao
                valor = preco
    
    #definindo o lucro potencial da carteira(posicao)
    if valor!=-10:
        valor_cheio = valor * pstd + pmean  #valor posicionado atual      
        dp = (preco * pstd + pmean) - valor_cheio #variacao do preco atual e do preco de compra/venda
        posicao = ncont * dp * 10 - custo*abs(ncont)           #posicao = lucro (POTENCIAL)
        
    return ncont, valor, posicao, reward

def obter_acao(ncont):
    decisao = modelo.toma_acao() #calcula a saida da rede neural
    return (decisao - lim_cont)

def rodar_1dia(precos, custo, dia):
    global melhor_reward
    ncont = 0 #cria variavel de quantidade de contratos
    ncont_anterior = 0 #cria variavel para quantidade de contratos anterior
    valor = -10 #cria variavel para preo medio
    reward = 0. #cria variavel para recompensa
    lucro = 0.
    posicao = 0 #cria variavel de posio 
    erro = []
    modelo.limpa_memoria_dia() #limpa o vetor de memoria
    done = False
    shp = modelo.janela_precos.shape
    
    for step in range( steps[dia - 1], steps[dia] ):  #roda os dados
        
        ultimos_precos = precos[ step : step + 1 ] #pega os valores de agora
        modelo.janela_precos = np.reshape(np.delete(np.insert(modelo.janela_precos,shp,np.concatenate(ultimos_precos.values).tolist()),[0,l]), (1,modelo.mkt_size))
        modelo.state = np.reshape([ncont/lim_cont, valor], (1,modelo.state_size)) #adiciona na variavel de estado      
        
        acao = obter_acao(ncont)        
        ncont, valor, posicao, reward = atuacao(precos['preco'][step], ncont, acao, custo, valor)
        
        #pos acao
        lucro += reward    #LUCRO ACUMULADO
        
        #v=valor if valor==-10 else valor*pstd+pmean
        #p=precos['preco'][step]*pstd+pmean
        #print("acao={0}; ncont={1}; valor={2}; preco={3}; posicao={4}; lucro={5}; reward={6}".format(acao, ncont, v, p, posicao, lucro, reward))    
            
    #fim do dia
    lucro += posicao    
    
    if lucro > melhor_reward:
        melhor_reward = lucro
    return lucro #retorna o valor do reward

dias_pos = 0
dias_neg = 0

def rodar_dias(precos, custo):   
    global dias_pos
    global dias_neg
    
    sum_rewards = 0 #cria variavel de somatoria de recompensas
    
    for dia in range( len(dias_para_rodar)-dias_teste+1, len(dias_para_rodar) ): #loop de dias
        reward = rodar_1dia(precos, custo, dia)
        print("Dia {0} resultado: R$ {1:0.2f}\n".format(dia, reward))
        sum_rewards += reward #roda 1 dia e adiciona o total na variavel de somatoria
        
        if reward>0: 
            dias_pos += 1
        elif reward<0:
            dias_neg += 1
    return sum_rewards


In [17]:
import time
##################################  MAIN  #########################################
if __name__ == "__main__":
    global dias_pos
    global dias_neg
    try:
        modelo.carrega_pesos('./pesos2.h5')
        modelo.limpa_memoria_epoca()
        sum_rewards = rodar_dias(inputs, custo) #adiciona o resultado da epoca na somatoria
        print("resultado do teste = {0:0.2f}".format(sum_rewards))
        print(dias_pos)
        print(dias_neg)
    finally:
        print("Melhor resultado diario: {0:0.2f}".format(melhor_reward))

W1107 21:59:41.427577  5600 deprecation_wrapper.py:119] From D:\anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Dia 229 resultado: R$ -282.40

Dia 230 resultado: R$ -770.74

Dia 231 resultado: R$ -54.82

Dia 232 resultado: R$ -1496.16

Dia 233 resultado: R$ -873.28

Dia 234 resultado: R$ 216.54

Dia 235 resultado: R$ -849.80

Dia 236 resultado: R$ -919.20

Dia 237 resultado: R$ -1325.74

Dia 238 resultado: R$ -1547.08

Dia 239 resultado: R$ -241.80

Dia 240 resultado: R$ -191.34

Dia 241 resultado: R$ -185.74

Dia 242 resultado: R$ 666.68

Dia 243 resultado: R$ -961.20

Dia 244 resultado: R$ -261.20

Dia 245 resultado: R$ -336.20

Dia 246 resultado: R$ -82.26

Dia 247 resultado: R$ 164.26

Dia 248 resultado: R$ 1593.80

Dia 249 resultado: R$ -681.20

Dia 250 resultado: R$ -171.20

Dia 251 resultado: R$ -26.80

Dia 252 resultado: R$ -917.26

Dia 253 resultado: R$ 483.20

Dia 254 resultado: R$ -252.26

Dia 255 resultado: R$ -176.80

Dia 256 resultado: R$ -106.80

Dia 257 resultado: R$ -1862.26

Dia 258 resultado: R$ -923.90

Dia 259 resultado: R$ 82.60

Dia 260 resultado: R$ 1349.78

Dia 261 resul